In [1]:
import requests, re
from bs4 import BeautifulSoup
from IPython.display import HTML, display

import pandas as pd

# https://www.cancerresearchuk.org/about-cancer/cancer-chat/talk-about-cancer/coping-with-loss
# https://www.cancerresearchuk.org/about-cancer/cancer-chat/talk-about-cancer/dying-with-cancer

# goes to page 278
# goes to page 247

# first page of thread:
# store data in lists --> pandas dataframe

# THREAD NAME
# <h3 class="post-title"><a href="/about-cancer/cancer-chat/thread/i-cant-live-without-my-dad-anyone-feel-the-same#post-1305941">I can&#039;t live without my dad - anyone feel the same?</a></h3>

# THREAD ADDRESS
# page_link

# USERNAME
# <div class="field field-name-ds-user-picture field-type-ds field-label-hidden">
# <div class="field-items"><div class="field-item even">
# <a href="/about-cancer/cancer-chat/users/paulus">

# POST DATE
# <span class="post-created hidden-xs">22 Jun 2020 16:35</span>

# POST TEXT
# <div class="field field-name-field-harmony-text field-type-text-long field-label-hidden">

# POST IS A REPLY TO POST_ID
# <span class="post-is-reply-to">22 Jun 2020 21:04 in response to <a href="/about-cancer/cancer-chat/thread/i-cant-live-without-my-dad-anyone-feel-the-same#post-1305941" class="active">bras1548</a></span>

# POST ID
# <a name="post-1305941"></a>


#### FUNCTION: SCRAPE A THREAD PAGE

In [2]:
# FUNCTION: SCRAPE A THREAD PAGE
def page_scraper_cancer_research(page_link):

    req = requests.get(page_link)
    soup = BeautifulSoup(req.text, 'lxml')
    
    # THREAD ADDRESS 
    thread_address_list = []
    thread_address = soup.find("link",{"rel":"canonical"})['href']
    thread_address = page_link
    #print(thread_address)

    # THREAD NAME
    thread_name_list = []
    thread_group = soup.find("h3", {"class": "post-title"})
    thread_name = thread_group.find("a").text
    #print(thread_name)

    # POST ID
    post_id_list = []
    post_id_group = soup.findAll("a", {"name": re.compile(r".*")})

    for post_id_g in post_id_group:
        if "post-" in post_id_g["name"]:
            post_id_list.append(post_id_g["name"])
            #print(post_id_g["name"])

    # POST USERNAME
    username_list = []
    user_group = soup.findAll("div", {"class": "field field-name-ds-user-picture field-type-ds field-label-hidden"})
    
    for user_g in user_group:    
        username = user_g.find("a")["href"]
        username = username.replace("/about-cancer/cancer-chat/users/", "")
        username_list.append(username)

        # populate THREAD ADDRESS, THREAD NAME lists
        thread_address_list.append(thread_address)
        thread_name_list.append(thread_name)
        #print(username)

    # POST DATE
    date_list = []
    date_group = soup.findAll("span", {"class": "post-created hidden-xs"})

    for date_g in date_group:
        date = date_g.text
        date_list.append(date)
        #print(date)

    # POST TEXT
    text_list = []
    text_group = soup.findAll("div", {"class": "field field-name-field-harmony-text field-type-text-long field-label-hidden"})

    for text_g in text_group:
        text_tag = text_g.find("div").find("div")
        text = text_tag.text
        text_list.append(text)
        #print(text_tag.text, "\n")

    # POST IS A REPLY TO
    reply_to_list = []
    reply_group = soup.findAll("span", {"class": "post-is-reply-to"})

    # add "N/A only to first posts in thread
    
    reply_group_len = len(reply_group)
    text_group_len = len(text_group)
    
    while reply_group_len < text_group_len:  # if CHANGED TO while
        reply_to_list.append("N/A")
        reply_group_len += 1
        
    for reply_g in reply_group:
        reply_to_id = reply_g.find("a")['href']
        reply_to_id = "https://www.cancerresearchuk.org" + reply_to_id
        reply_to_id = reply_to_id.replace(thread_address + "#", "")
        reply_to_list.append(reply_to_id)
        #print(reply_to_id)

    # COLUMN LISTS INTO A LIST OF ROWS
    new_rows = []

    for j in range(len(thread_name_list)):
        row_list = [thread_name_list[j], thread_address_list[j], username_list[j],
                    post_id_list[j], date_list[j], text_list[j], reply_to_list[j]]

        new_rows.append(row_list)
    
    return new_rows



#### FUNCTION: CHECK FOR "NEXT" PAGE IN THREAD

In [3]:
# FUNCTION: CHECK FOR "NEXT" PAGE IN THREAD

# <li class="cr-pagination__next">
# <a href="/about-cancer/cancer-chat/thread/i-cant-live-without-my-dad-anyone-feel-the-same?page=1">Next</a></li>

def next_page_cancer_research(page_link):

    req = requests.get(page_link)
    soup = BeautifulSoup(req.text, 'lxml')

    next_link = soup.find("li", {"class":"cr-pagination__next"})

    if next_link:
        next_link = next_link.find("a")['href']
        next_link = "https://www.cancerresearchuk.org/" + next_link
        #print(next_link)
        return next_link

    else:
        #print("last page")
        return None

print(next_page_cancer_research("https://www.cancerresearchuk.org/about-cancer/cancer-chat/thread/the-end-of-the-road?page=1"))
print(next_page_cancer_research("https://www.cancerresearchuk.org/about-cancer/cancer-chat/thread/the-end-of-the-road"))



None
https://www.cancerresearchuk.org//about-cancer/cancer-chat/thread/the-end-of-the-road?page=1


In [4]:
# TEST PAGE SCRAPER FUNCTION

page_link = "https://www.cancerresearchuk.org/about-cancer/cancer-chat/thread/the-end-of-the-road"
new_rows = page_scraper_cancer_research(page_link)


In [5]:
for row in new_rows:
    print(row[0])
    print(row[1])
    print(row[2])
    print(row[3])
    print(row[4])
    print(row[5])
    print(row[6], "\n")

The end of the road
https://www.cancerresearchuk.org/about-cancer/cancer-chat/thread/the-end-of-the-road
goodhubby
post-1458971
19 Dec 2020 10:49
I always hoped I would never be writing this but my wife was given the news last week that her metastatic breast cancer was no longer treatable. Seven years ago she was diagnosed with stage 1b HER + tumour in her right breast which was surgically removed followed by radiotherapy and chemo, immunotherapy etc then at 4 years 10 months she was diagnosed with brain mets which were removed followed by radio therapy an chemo again, this year in April the scans showed liver mets and a small one in her lung. The hard part for me was to watch her look so beautiful again and then have to go through the same over again. The liver mets would not respond to any chemo of which 3 were tried and it was only when my wife begged the consultant to try another that she was offered it but we were told that it won't work due to the last few not working.  Her liver

#### FUNCTION: SCRAPE ONE THREAD

In [11]:
# FUNCTION: SCRAPE ONE THREAD

row_list = []
page_link = "https://www.cancerresearchuk.org/about-cancer/cancer-chat/thread/the-end-of-the-road"

def thread_scraper_cancer_research(page_link):

    new_rows = page_scraper_cancer_research(page_link)
    next_link = next_page_cancer_research(page_link)
    
    #for row in new_rows:  # REMOVE THIS LATER
    #    print(row[0])
    #    print(row[1])
    #    print(row[2])
    #    print(row[3])
    #    print(row[4])
    #    print(row[5])
    #    print(row[6], "\n")
        
    
    if next_link is not None:
        new_rows.extend(thread_scraper_cancer_research(next_link))

    return new_rows

In [12]:
# TEST: SCRAPE ONE THREAD

page_link = "https://www.cancerresearchuk.org/about-cancer/cancer-chat/thread/the-end-of-the-road"

new_rows = thread_scraper_cancer_research(page_link)
print(len(new_rows))

26


#### FUNCTION: GET THREAD LINKS FROM PAGE

In [13]:
# FUNCTION: GET THREAD LINKS FROM PAGE

forum_link = "https://www.cancerresearchuk.org/about-cancer/cancer-chat/thread/the-end-of-the-road"

def thread_link_scraper_cancer_research(forum_link):

    thread_links = []
    req = requests.get(forum_link)
    soup = BeautifulSoup(req.text, 'lxml')

    # <div class="tl-cell views-field views-field-nothing" >
    # <a href="/about-cancer/cancer-chat/thread/has-anyone-tried-bereavement-counselling">

    thread_link_group = soup.find_all("div", {"class":"tl-cell views-field views-field-nothing"})
    
    for thread_link_gr in thread_link_group:    
        new_thread_link = thread_link_gr.find('a')['href']
        thread_links.append("https://www.cancerresearchuk.org" + new_thread_link)
        #print("https://www.cancerresearchuk.org" + new_thread_link, "\n")
        
    return thread_links
    


#### FUNCTION: CHECK FOR "NEXT" PAGE IN FORUM

In [14]:
# FUNCTION: CHECK FOR "NEXT" PAGE IN FORUM

def next_forum_page_cancer_research(forum_link):
    
    req = requests.get(forum_link)
    soup = BeautifulSoup(req.text, 'lxml')

# <li class="next last"><a title="Go to next page" href="
    next_link = soup.find("li", {"class":"next last"})

    if next_link:
        next_link = next_link.find("a")['href']
        next_link = "https://www.cancerresearchuk.org" + next_link
        #print(next_link)
        return next_link

    else:
        #print("last page")
        return None

print(next_forum_page_cancer_research("https://www.cancerresearchuk.org/about-cancer/cancer-chat/talk-about-cancer/dying-with-cancer"))
print(next_forum_page_cancer_research("https://www.cancerresearchuk.org/about-cancer/cancer-chat/talk-about-cancer/dying-with-cancer?page=247"))



https://www.cancerresearchuk.org/about-cancer/cancer-chat/talk-about-cancer/dying-with-cancer?page=1
None


#### FUNCTION: SCRAPE ALL THREAD LINKS ON FORUM

In [16]:
# FUNCTION: SCRAPE ALL THREAD LINKS ON FORUM

def thread_links_cancer_research(forum_link):
    
    thread_links = []
    current_page = forum_link
    new_links = thread_link_scraper_cancer_research(current_page)
    thread_links.extend(new_links)
    
    next_link = next_forum_page_cancer_research(current_page)
    print(next_link)
    
    if next_link is not None:
        thread_links.extend(thread_links_cancer_research(next_link))
        
    return thread_links




In [17]:
# TEST FUNCTION: SCRAPE ALL THREAD LINKS ON FORUM

forum_link = "https://www.cancerresearchuk.org/about-cancer/cancer-chat/talk-about-cancer/dying-with-cancer"
thread_links = thread_links_cancer_research(forum_link)


https://www.cancerresearchuk.org/about-cancer/cancer-chat/talk-about-cancer/dying-with-cancer?page=1
https://www.cancerresearchuk.org/about-cancer/cancer-chat/talk-about-cancer/dying-with-cancer?page=2
https://www.cancerresearchuk.org/about-cancer/cancer-chat/talk-about-cancer/dying-with-cancer?page=3
https://www.cancerresearchuk.org/about-cancer/cancer-chat/talk-about-cancer/dying-with-cancer?page=4
https://www.cancerresearchuk.org/about-cancer/cancer-chat/talk-about-cancer/dying-with-cancer?page=5
https://www.cancerresearchuk.org/about-cancer/cancer-chat/talk-about-cancer/dying-with-cancer?page=6
https://www.cancerresearchuk.org/about-cancer/cancer-chat/talk-about-cancer/dying-with-cancer?page=7
https://www.cancerresearchuk.org/about-cancer/cancer-chat/talk-about-cancer/dying-with-cancer?page=8
https://www.cancerresearchuk.org/about-cancer/cancer-chat/talk-about-cancer/dying-with-cancer?page=9
https://www.cancerresearchuk.org/about-cancer/cancer-chat/talk-about-cancer/dying-with-canc

https://www.cancerresearchuk.org/about-cancer/cancer-chat/talk-about-cancer/dying-with-cancer?page=82
https://www.cancerresearchuk.org/about-cancer/cancer-chat/talk-about-cancer/dying-with-cancer?page=83
https://www.cancerresearchuk.org/about-cancer/cancer-chat/talk-about-cancer/dying-with-cancer?page=84
https://www.cancerresearchuk.org/about-cancer/cancer-chat/talk-about-cancer/dying-with-cancer?page=85
https://www.cancerresearchuk.org/about-cancer/cancer-chat/talk-about-cancer/dying-with-cancer?page=86
https://www.cancerresearchuk.org/about-cancer/cancer-chat/talk-about-cancer/dying-with-cancer?page=87
https://www.cancerresearchuk.org/about-cancer/cancer-chat/talk-about-cancer/dying-with-cancer?page=88
https://www.cancerresearchuk.org/about-cancer/cancer-chat/talk-about-cancer/dying-with-cancer?page=89
https://www.cancerresearchuk.org/about-cancer/cancer-chat/talk-about-cancer/dying-with-cancer?page=90
https://www.cancerresearchuk.org/about-cancer/cancer-chat/talk-about-cancer/dying-

https://www.cancerresearchuk.org/about-cancer/cancer-chat/talk-about-cancer/dying-with-cancer?page=162
https://www.cancerresearchuk.org/about-cancer/cancer-chat/talk-about-cancer/dying-with-cancer?page=163
https://www.cancerresearchuk.org/about-cancer/cancer-chat/talk-about-cancer/dying-with-cancer?page=164
https://www.cancerresearchuk.org/about-cancer/cancer-chat/talk-about-cancer/dying-with-cancer?page=165
https://www.cancerresearchuk.org/about-cancer/cancer-chat/talk-about-cancer/dying-with-cancer?page=166
https://www.cancerresearchuk.org/about-cancer/cancer-chat/talk-about-cancer/dying-with-cancer?page=167
https://www.cancerresearchuk.org/about-cancer/cancer-chat/talk-about-cancer/dying-with-cancer?page=168
https://www.cancerresearchuk.org/about-cancer/cancer-chat/talk-about-cancer/dying-with-cancer?page=169
https://www.cancerresearchuk.org/about-cancer/cancer-chat/talk-about-cancer/dying-with-cancer?page=170
https://www.cancerresearchuk.org/about-cancer/cancer-chat/talk-about-canc

https://www.cancerresearchuk.org/about-cancer/cancer-chat/talk-about-cancer/dying-with-cancer?page=242
https://www.cancerresearchuk.org/about-cancer/cancer-chat/talk-about-cancer/dying-with-cancer?page=243
https://www.cancerresearchuk.org/about-cancer/cancer-chat/talk-about-cancer/dying-with-cancer?page=244
https://www.cancerresearchuk.org/about-cancer/cancer-chat/talk-about-cancer/dying-with-cancer?page=245
https://www.cancerresearchuk.org/about-cancer/cancer-chat/talk-about-cancer/dying-with-cancer?page=246
https://www.cancerresearchuk.org/about-cancer/cancer-chat/talk-about-cancer/dying-with-cancer?page=247
None


In [19]:
# OPEN FILE WITH SAVED 'DYING WITH CANCER' LINKS

thread_links = pd.read_csv(r'/Users/Sofia/Desktop/cancer_research_dying_cancer_links.csv', delimiter="\t")
thread_links = thread_links["0"].values.tolist()
print(len(thread_links))


#test_list = [0, 1, 2, 3, 4, 5, 6, 7, 8, 9]

#for i in test_list[0:3]:
#    print(i)
    
#print("\n")

#for i in test_list[3:6]:
#    print(i)
    
#print("\n")

#for i in test_list[6:]:
#    print(i)

2475


In [108]:
# DEBUGGING PROBLEM PAGE

bug_link = "https://www.cancerresearchuk.org/about-cancer/cancer-chat/thread/confused-30"
#bug_link = "https://www.cancerresearchuk.org/about-cancer/cancer-chat/thread/seeing-signs-from-deceased-loved-ones?page=7"

bug_rows = thread_scraper_cancer_research(bug_link)
#bug_rows = page_scraper_cancer_research(bug_link)

In [110]:
#for line in bug_rows:
#    print(line[0])
#    print(line[1])
#    print(line[2])
#    print(line[3])
#    print(line[4])
#    print(line[5])
#    print(line[6], "\n")

In [34]:
# RUN 'SCRAPE THREAD' ON THREAD_LINKS

#dying_cancer_data = [] # DO NOT RESET !!

count = 1668

for thread in thread_links[1667:]:
    
    print(count, thread)

    new_rows = thread_scraper_cancer_research(thread)    
    dying_cancer_data.extend(new_rows)
    
    count += 1



1668 https://www.cancerresearchuk.org/about-cancer/cancer-chat/thread/mum-has-been-given-a-week
1669 https://www.cancerresearchuk.org/about-cancer/cancer-chat/thread/dad-dying-my-mom-doesnt-care-about-me-im-alone
1670 https://www.cancerresearchuk.org/about-cancer/cancer-chat/thread/mum-diagnosed-with-metastatic-cancer-terminal
1671 https://www.cancerresearchuk.org/about-cancer/cancer-chat/thread/my-dad-is-dying-3
1672 https://www.cancerresearchuk.org/about-cancer/cancer-chat/thread/to-much
1673 https://www.cancerresearchuk.org/about-cancer/cancer-chat/thread/please-help-me-3
1674 https://www.cancerresearchuk.org/about-cancer/cancer-chat/thread/why-do-they-want-to-increase-my-dads-appetite
1675 https://www.cancerresearchuk.org/about-cancer/cancer-chat/thread/my-dad-has-cancer-and-i-feel-like-im-watching-him-die
1676 https://www.cancerresearchuk.org/about-cancer/cancer-chat/thread/mum-is-slowly-going-and-i-have-been-asked-to-keep-quiet
1677 https://www.cancerresearchuk.org/about-cancer/c

1751 https://www.cancerresearchuk.org/about-cancer/cancer-chat/thread/feel-so-lonely
1752 https://www.cancerresearchuk.org/about-cancer/cancer-chat/thread/my-very-sad-loss-2
1753 https://www.cancerresearchuk.org/about-cancer/cancer-chat/thread/my-brave-dad
1754 https://www.cancerresearchuk.org/about-cancer/cancer-chat/thread/my-mom-has-gone
1755 https://www.cancerresearchuk.org/about-cancer/cancer-chat/thread/i-dont-know-if-i-can-cope-with-it-now
1756 https://www.cancerresearchuk.org/about-cancer/cancer-chat/thread/mom-is-in-her-final-days
1757 https://www.cancerresearchuk.org/about-cancer/cancer-chat/thread/my-mom-died-less-than-a-week-after-being-diagnosed
1758 https://www.cancerresearchuk.org/about-cancer/cancer-chat/thread/dad-has-three-months-left-at-best
1759 https://www.cancerresearchuk.org/about-cancer/cancer-chat/thread/my-wonderful-sister-has-had-the-news-her-cancer-has-returned
1760 https://www.cancerresearchuk.org/about-cancer/cancer-chat/thread/my-mum-passed-away-6-months-

1833 https://www.cancerresearchuk.org/about-cancer/cancer-chat/thread/just-want-to-talk
1834 https://www.cancerresearchuk.org/about-cancer/cancer-chat/thread/final-days-of-lung-cancer
1835 https://www.cancerresearchuk.org/about-cancer/cancer-chat/thread/end-of-life-care
1836 https://www.cancerresearchuk.org/about-cancer/cancer-chat/thread/its-becoming-scarily-real
1837 https://www.cancerresearchuk.org/about-cancer/cancer-chat/thread/this-is-inspiration
1838 https://www.cancerresearchuk.org/about-cancer/cancer-chat/thread/stage-4-lung-cancer
1839 https://www.cancerresearchuk.org/about-cancer/cancer-chat/thread/stomach-cancer
1840 https://www.cancerresearchuk.org/about-cancer/cancer-chat/thread/im-going-to-loose-my-mum-soon
1841 https://www.cancerresearchuk.org/about-cancer/cancer-chat/thread/my-aunt-has-days-left
1842 https://www.cancerresearchuk.org/about-cancer/cancer-chat/thread/is-all-of-this-worth-it
1843 https://www.cancerresearchuk.org/about-cancer/cancer-chat/thread/dying-from-a

1918 https://www.cancerresearchuk.org/about-cancer/cancer-chat/thread/my-dad-was-diagnosed-with-pancreatic-cancer-in-april
1919 https://www.cancerresearchuk.org/about-cancer/cancer-chat/thread/dying-young-alone-why
1920 https://www.cancerresearchuk.org/about-cancer/cancer-chat/thread/my-moma-is-going-to-die
1921 https://www.cancerresearchuk.org/about-cancer/cancer-chat/thread/really-need-someone-to-talk-to
1922 https://www.cancerresearchuk.org/about-cancer/cancer-chat/thread/looking-for-companion
1923 https://www.cancerresearchuk.org/about-cancer/cancer-chat/thread/i-am-losing-my-best-friend-my-dad
1924 https://www.cancerresearchuk.org/about-cancer/cancer-chat/thread/please-help-me-understand-my-fathers-death
1925 https://www.cancerresearchuk.org/about-cancer/cancer-chat/thread/widowers-forum-for-those-who-might-be-interested
1926 https://www.cancerresearchuk.org/about-cancer/cancer-chat/thread/being-terminal-my-cure-for-captain-fantastic
1927 https://www.cancerresearchuk.org/about-can

1996 https://www.cancerresearchuk.org/about-cancer/cancer-chat/thread/so-much-life-to-live
1997 https://www.cancerresearchuk.org/about-cancer/cancer-chat/thread/bering-terminal-part-5-hard-core-truth-acceptance-vs-immortal-vs-remembered-vs-never-left
1998 https://www.cancerresearchuk.org/about-cancer/cancer-chat/thread/awwww-thanks-being-terminal-part-4
1999 https://www.cancerresearchuk.org/about-cancer/cancer-chat/thread/slowly-losing-my-girlfriend
2000 https://www.cancerresearchuk.org/about-cancer/cancer-chat/thread/my-husband/of35yrs/soulmate/passedaway
2001 https://www.cancerresearchuk.org/about-cancer/cancer-chat/thread/being-terminal-part-2-last-topic-in-the-forum-section-lets-pick-a-fight
2002 https://www.cancerresearchuk.org/about-cancer/cancer-chat/thread/dying-of-pancreatic-cancer
2003 https://www.cancerresearchuk.org/about-cancer/cancer-chat/thread/my-mum-died-when-i-was-one
2004 https://www.cancerresearchuk.org/about-cancer/cancer-chat/thread/alone-and-dying
2005 https://ww

2076 https://www.cancerresearchuk.org/about-cancer/cancer-chat/thread/dying-with-lung-cancer
2077 https://www.cancerresearchuk.org/about-cancer/cancer-chat/thread/hope-3yrs-ago-i-was-given-6-weeks-to-live
2078 https://www.cancerresearchuk.org/about-cancer/cancer-chat/thread/one-day-at-a-time
2079 https://www.cancerresearchuk.org/about-cancer/cancer-chat/thread/someone-to-chat-with
2080 https://www.cancerresearchuk.org/about-cancer/cancer-chat/thread/my-beautiful-mom-0
2081 https://www.cancerresearchuk.org/about-cancer/cancer-chat/thread/being-the-one-left-behind
2082 https://www.cancerresearchuk.org/about-cancer/cancer-chat/thread/where-will-i-end
2083 https://www.cancerresearchuk.org/about-cancer/cancer-chat/thread/dont-know-how-im-going-to-cope
2084 https://www.cancerresearchuk.org/about-cancer/cancer-chat/thread/in-need-of-support-please
2085 https://www.cancerresearchuk.org/about-cancer/cancer-chat/thread/sad-times-again
2086 https://www.cancerresearchuk.org/about-cancer/cancer-cha

2160 https://www.cancerresearchuk.org/about-cancer/cancer-chat/thread/sister-on-steriods
2161 https://www.cancerresearchuk.org/about-cancer/cancer-chat/thread/my-mum-has-pancreatic-cancer-1
2162 https://www.cancerresearchuk.org/about-cancer/cancer-chat/thread/my-mom
2163 https://www.cancerresearchuk.org/about-cancer/cancer-chat/thread/how-do-i-delete-this-post
2164 https://www.cancerresearchuk.org/about-cancer/cancer-chat/thread/my-children-are-losing-their-dad-0
2165 https://www.cancerresearchuk.org/about-cancer/cancer-chat/thread/new-to-all-this
2166 https://www.cancerresearchuk.org/about-cancer/cancer-chat/thread/my-beautiful-kate
2167 https://www.cancerresearchuk.org/about-cancer/cancer-chat/thread/so-sad
2168 https://www.cancerresearchuk.org/about-cancer/cancer-chat/thread/grandfather-not-coping-well-with-terminal-diagnosis
2169 https://www.cancerresearchuk.org/about-cancer/cancer-chat/thread/just-been-told-my-nan-has-a-few-months-to-live
2170 https://www.cancerresearchuk.org/abou

2242 https://www.cancerresearchuk.org/about-cancer/cancer-chat/thread/rectal-cancer-and-liver-mets
2243 https://www.cancerresearchuk.org/about-cancer/cancer-chat/thread/losing-a-loved-one-0
2244 https://www.cancerresearchuk.org/about-cancer/cancer-chat/thread/dad-has-not-got-long-left
2245 https://www.cancerresearchuk.org/about-cancer/cancer-chat/thread/shes-gone
2246 https://www.cancerresearchuk.org/about-cancer/cancer-chat/thread/my-mums-brave-battle
2247 https://www.cancerresearchuk.org/about-cancer/cancer-chat/thread/where-will-my-grandad-go
2248 https://www.cancerresearchuk.org/about-cancer/cancer-chat/thread/dont-really-know-where-to-start
2249 https://www.cancerresearchuk.org/about-cancer/cancer-chat/thread/help-with-terminal-luekamia
2250 https://www.cancerresearchuk.org/about-cancer/cancer-chat/thread/2186
2251 https://www.cancerresearchuk.org/about-cancer/cancer-chat/thread/hello-2
2252 https://www.cancerresearchuk.org/about-cancer/cancer-chat/thread/lung-cancer-and-coughing-

2327 https://www.cancerresearchuk.org/about-cancer/cancer-chat/thread/scared-3
2328 https://www.cancerresearchuk.org/about-cancer/cancer-chat/thread/23-with-metastatic-cancer
2329 https://www.cancerresearchuk.org/about-cancer/cancer-chat/thread/diagnosis-to-death-in-24-hours-cant-comprehend-it
2330 https://www.cancerresearchuk.org/about-cancer/cancer-chat/thread/1327
2331 https://www.cancerresearchuk.org/about-cancer/cancer-chat/thread/leiomyosarcoma
2332 https://www.cancerresearchuk.org/about-cancer/cancer-chat/thread/x-0
2333 https://www.cancerresearchuk.org/about-cancer/cancer-chat/thread/my-mum-is-dying-i-am-trying-to-be-strongand-will-continue-to-do-so
2334 https://www.cancerresearchuk.org/about-cancer/cancer-chat/thread/im-20-years-old-and-my-mum-is-dying
2335 https://www.cancerresearchuk.org/about-cancer/cancer-chat/thread/hes-still-with-us
2336 https://www.cancerresearchuk.org/about-cancer/cancer-chat/thread/how-do-i-tell-my-son
2337 https://www.cancerresearchuk.org/about-cance

2413 https://www.cancerresearchuk.org/about-cancer/cancer-chat/thread/my-husband-kevin-died-on-the-18th-of-march-his-last-days-were-spent-in-an-hospice
2414 https://www.cancerresearchuk.org/about-cancer/cancer-chat/thread/hi-im-new-on-here-my-partner-has-terminal-cancer-was-given-6-months-2years-last-june
2415 https://www.cancerresearchuk.org/about-cancer/cancer-chat/thread/kidney-cancer-0
2416 https://www.cancerresearchuk.org/about-cancer/cancer-chat/thread/my-dad-dying-of-cancer
2417 https://www.cancerresearchuk.org/about-cancer/cancer-chat/thread/realised-i-am-terminalwhat-next
2418 https://www.cancerresearchuk.org/about-cancer/cancer-chat/thread/what-to-eat-with-terminal-brain-cancer
2419 https://www.cancerresearchuk.org/about-cancer/cancer-chat/thread/chemotherapy-my-dad-has-small-cell-lung-cancer-we-are-all-so-very-scared
2420 https://www.cancerresearchuk.org/about-cancer/cancer-chat/thread/my-dad-and-now-my-mum
2421 https://www.cancerresearchuk.org/about-cancer/cancer-chat/threa

In [36]:
# SAVE PRELIMINARY DATA

column_names = ['Thread_name', 'Thread_address', 'Username', "Post_id",
                                  "Date", "Post_text", "Reply_to_id"]

cancer_research_dying_cancer_prelim = pd.DataFrame(dying_cancer_data, columns = column_names)  

cancer_research_dying_cancer_prelim.to_csv(r'/Users/Sofia/Desktop/cancer_research_dying_cancer.csv', sep ='\t', index = False, header=True)




In [35]:
print(len(dying_cancer_data))

# print(len(thread_links))
    
#for line in dying_cancer_data:
#    print(line[0])
#    print(line[1])
#    print(line[2])
#    print(line[3])
#    print(line[4])
#    print(line[5])
#    print(line[6], "\n")


17792


In [25]:
# SAVE 'DYING WITH CANCER' FORUM LINKS TO FILE

#print(len(thread_links))

#for thread in thread_links:
#    print(thread)

cancer_research_dying_cancer_links = pd.DataFrame(thread_links)
cancer_research_dying_cancer_links.to_csv(r'/Users/Sofia/Desktop/cancer_research_dying_cancer_links.csv', sep ='\t', index = False, header=True)



In [137]:
# GLOBAL database for 
# https://www.cancerresearchuk.org/about-cancer/cancer-chat/talk-about-cancer/coping-with-loss

column_names = ['Thread_name', 'Thread_address', 'Username', "Post_id",
                                  "Date", "Post_text", "Reply_to_id"]

cancer_research_coping_w_loss = pd.DataFrame(new_rows, columns = column_names)  
print (cancer_research_coping_w_loss)

cancer_research_coping_w_loss.to_csv(r'/Users/Sofia/Desktop/cancer_research_coping_loss_test.csv', sep ='\t', index = False, header=True)


                                          Thread_name  \
0   I can't live without my dad - anyone feel the ...   
1   I can't live without my dad - anyone feel the ...   
2   I can't live without my dad - anyone feel the ...   
3   I can't live without my dad - anyone feel the ...   
4   I can't live without my dad - anyone feel the ...   
5   I can't live without my dad - anyone feel the ...   
6   I can't live without my dad - anyone feel the ...   
7   I can't live without my dad - anyone feel the ...   
8   I can't live without my dad - anyone feel the ...   
9   I can't live without my dad - anyone feel the ...   
10  I can't live without my dad - anyone feel the ...   
11  I can't live without my dad - anyone feel the ...   
12  I can't live without my dad - anyone feel the ...   
13  I can't live without my dad - anyone feel the ...   
14  I can't live without my dad - anyone feel the ...   
15  I can't live without my dad - anyone feel the ...   
16  I can't live without my dad